## Overall Workflow

![alt text](../img/download.png "Title")

- Two very important concepts in Elasticsearch are documents and indexes.

- A document is collection of fields with their associated values.

- To work with Elasticsearch you have to organize your data into documents, and then add all your documents to an index.

- Index as a collection of documents that is stored in a highly optimized format designed to perform efficient searches.

In [1]:
import json
from elasticsearch import Elasticsearch
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer
# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work

/Users/viviensiew/.local/share/virtualenvs/llm-zoomcamp-0DI9tvdQ/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 1: Prepare documents

In [2]:
# with open('documents.json', 'rt') as f_in:
#     docs_raw = json.load(f_in)

In [3]:
doc_json = "../pre-course workshop 1/documents.json"

with open(doc_json, 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
docs_raw

[{'course': 'data-engineering-zoomcamp',
  'documents': [{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?'},
   {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
    'section': 'General course-related questions',
    'question': 'Course - What are the prerequisites for this course?'},
   {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in

The documents to be indexed needs to be in a normalized format instead of nested format. Hence additional preprocessing is done below to normalise the documents.

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

### Step 2: Create Embeddings using Pretrained Models
Sentence Transformers documentation here: [https://www.sbert.net/docs/sentence_transformer/pretrained_models.html](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html)

- Allows usage of pretrained Sentence Transformers models via Sentence Transformers Hugging Face organization for sentence embeddings.
- [List of pretrained models](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#original-models).

In [6]:
model = SentenceTransformer("all-mpnet-base-v2")

In [7]:
# Example of plain text to vector embeddings
model.encode("This is a simple sentence")

array([ 4.44872258e-03, -7.61313960e-02, -3.77426128e-04,  7.52522098e-03,
       -3.80979255e-02,  3.80131267e-02, -9.73012485e-03, -5.05393837e-03,
       -9.37976502e-03,  1.23888440e-02,  4.91276719e-02,  1.52209392e-02,
        3.80008593e-02, -6.41803443e-02,  9.42136347e-03, -5.19748516e-02,
        9.08066705e-02,  1.71115343e-02,  1.62125435e-02,  2.98866928e-02,
        1.50534813e-03,  8.35076533e-03,  3.78842913e-02, -1.01192091e-02,
        6.46107364e-03,  3.97896656e-05, -1.45217581e-02, -1.88468546e-02,
       -3.74039784e-02, -1.51658291e-03, -1.02681359e-02, -3.68063040e-02,
        2.36677416e-02, -6.46022335e-02,  1.96967039e-06, -5.01107424e-03,
       -2.80826259e-03, -1.92073286e-02, -8.65120068e-02,  2.83464156e-02,
       -5.38666956e-02,  3.63704711e-02, -2.26467717e-02,  2.87367161e-02,
       -1.32342195e-02,  1.08689629e-01,  3.70519124e-02,  3.38801667e-02,
       -5.30679710e-02,  3.61782387e-02, -1.35722314e-03, -3.63483056e-02,
       -2.78346930e-02, -

In [8]:
len(model.encode("This is a simple sentence"))

768

In [9]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

As above, the fields "section" and "course" are categorical data which do not carry much information. Hence for this exercise, the fields "text" and "question" would be embedded instead.

In [10]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

### Step 3: Setup ElasticSearch connection

In [11]:
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'f0908f942491', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Zqofeq0fQJO25UCNdAtFmw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Step 4: Create Mappings and Index
Mapping is the process of defining how a document, and the fields it contains, are stored and indexed. 

Each document is a collection of fields, which each have their own data type.

We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored

Note that the index_settings are same as in previous modules except for "text_vector". The dims are taken from the dimensions of output from model.encode().

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector":{"type":"dense_vector","dims": 768,"index":True,"similarity": "cosine"
        },
        }
    }
}

Delete the index if it already exists before create it.

In [13]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Step 5: Add documents into index

In [14]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

### Step 6: Create end user query

Encode query into vector embeddings

In [15]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

Specify the search fields given a query.

- k: Number of nearest neighbours since it's using KNN.
- num_candidates: Num. of documents to look into.

In [16]:
query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

Query results are retrieved using a KNN search to find the nearest vectors. Note that these results contain scores (0-1) to indicate the similarity to the query.

In [17]:
res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'sEJcgpABHwzCw5qyCtEj',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'hEJcgpABHwzCw5qyE9Px',
  '_score': 0.60555583,
  '_source': {'question': "The answer I get for one of the homework questions doesn't match any of the options. What should I do?",
   'course': 'machine-learning-zoomcamp',
   'section': '2. Machine Learning for Regression',
   'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different versions of libraries — even different versions of Python.\nIf it’s the case, just select the option that’s closest to your answer'}},
 {'_index': 'course-questions',
  '

### Step 7: Perform Semantic Search & Advanced Search

In [18]:
response = es_client.search(
    index=index_name,
    query={
        "bool": {
            "must": {
             "multi_match": 
                        {"query": "windows or python?", 
                         "fields": ["text", "question","course","title"],
                         "type": "best_fields"
                        }
                    },
            "filter": {
                "term": {
                        "course": "data-engineering-zoomcamp"
            }
        }
        }
    }
)

In [19]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': '9kJcgpABHwzCw5qyC9Gb',
  '_score': 7.728908,
  '_source': {'text': 'Problem: If you have already installed pgcli but bash doesn\'t recognize pgcli\nOn Git bash: bash: pgcli: command not found\nOn Windows Terminal: pgcli: The term \'pgcli\' is not recognized…\nSolution: Try adding a Python path C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts to Windows PATH\nFor details:\nGet the location: pip list -v\nCopy C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\site-packages\n3. Replace site-packages with Scripts: C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts\nIt can also be that you have Python installed elsewhere.\nFor me it was under c:\\python310\\lib\\site-packages\nSo I had to add c:\\python310\\lib\\Scripts to PATH, as shown below.\nPut the above path in "Path" (or "PATH") in System Variables\nReference: https://stackoverflow.com/a/68233660',
   'section': 'Module 1: Docker and Terraform',
   'question': 'PGCLI - pg

### References
1. https://logz.io/blog/elasticsearch-mapping/#:~:text=Within%20a%20search%20engine%2C%20mapping,indexes%20and%20stores%20its%20fields

2. https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

3. https://www.elastic.co/search-labs/tutorials

4. https://www.elastic.co/search-labs/blog/text-similarity-search-with-vectors-in-elasticsearch